In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
#Progress bar fix: use callbacks=[TQDMNotebookCallback()] in model.fit
#verbose=0 is also required
import JupyterProgbarLogger as Logger
from tqdm import tqdm_notebook as tqdm
from kerastuner.tuners import RandomSearch

from time import time
import numpy as np
import random
import matplotlib.pyplot as plt

import h5py

In [2]:
BATCH_SIZE=32
DATA_AMOUNT=200000

In [3]:
file_path = "/home/amitp/Documents/Python/2019-notebooks/Action Recognition/images_raw_doric_round1.h5"
with h5py.File(file_path,'r') as f:
    labels = f['/labels'][:DATA_AMOUNT]
    data = f['/frames/raw'][:DATA_AMOUNT]
data = data[(labels>=0)]
labels = labels[(labels>=0)]
if len(data.shape) < 4:
    data=data[...,None]
train_data=data[:int(DATA_AMOUNT*3/4)]
train_labels=labels[:int(DATA_AMOUNT*3/4)]
val_data=data[int(DATA_AMOUNT*3/4):int(DATA_AMOUNT*7/8)]
val_labels=labels[int(DATA_AMOUNT*3/4):int(DATA_AMOUNT*7/8)]
test_data=data[int(DATA_AMOUNT*7/8):DATA_AMOUNT]
test_labels=labels[int(DATA_AMOUNT*7/8):DATA_AMOUNT]

In [4]:
# old-fashioned convnet for flip detection...
def build_model(
                input_shape=(80, 80, 1),
                stride_length=(1, 1),
                kernel=(4, 4),
                kernel_initializer='glorot_uniform',
                activation=layers.Activation('relu'),
                dense_activation=layers.Activation('relu'),
                output_activation=layers.Activation('softmax'),
                batch_momentum=.999,
                combine=True,
                padding='valid',
                batch_norm=False
            ):
    nfilters=[4,8,16,32]
    inputs = layers.Input(shape=input_shape)
    x = inputs

    conv_parameters = {
        'padding': padding,
        'strides': stride_length,
        'kernel_initializer': kernel_initializer
    }

    # encode net
    for filters in nfilters:
        x = layers.Conv2D(filters, kernel, **conv_parameters)(x)
        x = layers.BatchNormalization(momentum=batch_momentum)(x)
        x = activation(x)
        x = layers.MaxPooling2D((2, 2), padding=padding)(x)
    #x = layers.Conv2D(1, (1, 1))(x)
    #x = activation(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(46)(x)
    output = output_activation(x)

    model = keras.models.Model(inputs, output)
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=1e-4,
            momentum=0.9,
            nesterov=True,
            decay=1e-6
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [5]:
model = build_model()

In [6]:
history = model.fit(data,
                    labels,
                    epochs=2,
                    verbose=0,
                    validation_data=(val_data,val_labels),
                    callbacks=[Logger.JupyterProgbarLogger()]
                   )

W0716 12:05:22.942905 140486872340288 deprecation.py:323] From /home/amitp/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2


Epoch 2/2


In [7]:
model.evaluate(test_data,test_labels)

24994/24994 [==============================] - ETA: 12s - loss: 1.5624 - accuracy: 0.625 - ETA: 11s - loss: 1.9244 - accuracy: 0.475 - ETA: 11s - loss: 2.0868 - accuracy: 0.392 - ETA: 11s - loss: 1.6971 - accuracy: 0.519 - ETA: 11s - loss: 1.6993 - accuracy: 0.512 - ETA: 10s - loss: 1.7492 - accuracy: 0.531 - ETA: 10s - loss: 1.7946 - accuracy: 0.505 - ETA: 10s - loss: 1.8180 - accuracy: 0.495 - ETA: 10s - loss: 1.7859 - accuracy: 0.497 - ETA: 10s - loss: 1.8165 - accuracy: 0.468 - ETA: 9s - loss: 1.8295 - accuracy: 0.468 - ETA: 9s - loss: 1.8807 - accuracy: 0.45 - ETA: 9s - loss: 1.8974 - accuracy: 0.44 - ETA: 9s - loss: 1.9553 - accuracy: 0.41 - ETA: 9s - loss: 1.9909 - accuracy: 0.40 - ETA: 9s - loss: 1.9751 - accuracy: 0.41 - ETA: 9s - loss: 1.9893 - accuracy: 0.41 - ETA: 9s - loss: 1.9681 - accuracy: 0.42 - ETA: 9s - loss: 1.8806 - accuracy: 0.46 - ETA: 9s - loss: 1.8932 - accuracy: 0.45 - ETA: 9s - loss: 1.8637 - accuracy: 0.46 - ETA: 9s - loss: 1.8554 - accuracy: 0.46 - ETA: 9s 

[2.1438371247078423, 0.37933105]